In [1]:
import os
os.cpu_count()

8

In [2]:
import sqlalchemy
import logging


In [3]:
from testcontainers.core.container import DockerContainer
from testcontainers.core.waiting_utils import wait_for_logs

def nldi_db_container():
    dc = DockerContainer("ghcr.io/internetofwater/nldi-db:demo")
    dc.with_exposed_ports(5432)
    dc.with_env("POSTGRES_PASSWORD", "changeMe")
    dc.with_env("NLDI_DATABASE_ADDRESS", "localhost")
    dc.with_env("NLDI_DATABASE_NAME", "nldi")
    dc.with_env("NLDI_DB_OWNER_USERNAME", "nldi")
    dc.with_env("NLDI_DB_OWNER_PASSWORD", "changeMe")
    dc.with_env("NLDI_SCHEMA_OWNER_USERNAME", "nldi_schema_owner")
    dc.with_env("NLDI_SCHEMA_OWNER_PASSWORD", "changeMe")
    dc.with_env("NHDPLUS_SCHEMA_OWNER_USERNAME", "nhdplus")
    dc.with_env("NLDI_READ_ONLY_USERNAME", "read_only_user")
    dc.with_env("NLDI_READ_ONLY_PASSWORD", "changeMe")
    dc.with_volume_mapping("data", "/var/lib/postgresql/data", mode="rw")

    dc.start()
    delay = wait_for_logs(dc, "database system is ready to accept connections")
    logging.info(f"Postgres container started in {delay} seconds")

    client = dc.get_docker_client()
    db_info = {
        "user": "nldi",
        "password": "changeMe",
        "host": client.bridge_ip(dc.get_wrapped_container().short_id),
        "port": 5432,
        "dbname": "nldi",
    }
    return db_info



In [4]:
dc = nldi_db_container()

Pulling image testcontainers/ryuk:0.7.0
Container started: 16c8fba90940
Waiting for container <Container: 16c8fba90940> with image testcontainers/ryuk:0.7.0 to be ready ...
Pulling image ghcr.io/internetofwater/nldi-db:demo
Container started: f5d1fceca0d5


In [7]:
dc

{'user': 'nldi',
 'password': 'changeMe',
 'host': '172.18.0.3',
 'port': 5432,
 'dbname': 'nldi'}

In [ ]:






dc.stop()

In [21]:
engine = sqlalchemy.create_engine(f'postgresql+psycopg2://nldi:changeMe@{client.bridge_ip(dc.get_wrapped_container().short_id)}:5432/nldi')
with engine.begin() as connection:
    v, = connection.execute(sqlalchemy.text("SELECT 1")).fetchone()
print(v)

1


In [22]:
dc.stop()